In [3]:
import pandas as pd 
df=pd.read_csv('https://raw.githubusercontent.com/codebasics/py/master/ML/19_Bagging/Exercise/heart.csv')
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [4]:
df.isna().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [5]:
df['Sex']=df.Sex.map({'M':1,'F':0})
dum=pd.get_dummies(df['ChestPainType'])
dum=dum.drop('TA',axis='columns')
df1=pd.concat([df,dum],axis='columns')
y=df1.HeartDisease
df1=df1.drop(['ChestPainType','HeartDisease'],axis='columns')
df1.ExerciseAngina=df1.ExerciseAngina.map({'N':0,'Y':1})
df1.ST_Slope=df1.ST_Slope.replace({'Up':3,'Flat':2,'Down':1})
df1.RestingECG.replace({'Normal':1,'ST':2,'LVH':3},inplace=True)
df1


,Age,Sex,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,ASY,ATA,NAP
0,40,1,140,289,0,1,172,0,0.0,3,0,1,0
1,49,0,160,180,0,1,156,0,1.0,2,0,0,1
2,37,1,130,283,0,2,98,0,0.0,3,0,1,0
3,48,0,138,214,0,1,108,1,1.5,2,1,0,0
4,54,1,150,195,0,1,122,0,0.0,3,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,1,110,264,0,1,132,0,1.2,2,0,0,0
914,68,1,144,193,1,1,141,0,3.4,2,1,0,0
915,57,1,130,131,0,1,115,1,1.2,2,1,0,0
916,57,0,130,236,0,3,174,0,0.0,2,0,1,0


In [10]:
from sklearn.preprocessing import MinMaxScaler
mn=MinMaxScaler()
x_scale=mn.fit_transform(df1)

In [11]:
from sklearn.svm import SVC
svm_m=SVC()


In [16]:
from sklearn.model_selection import cross_val_score
score=cross_val_score(svm_m,x_scale,y,cv=10)
score.mean()

0.8407907310081223

In [22]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_scale,y,test_size=0.2,stratify=y,)

In [24]:
from sklearn.ensemble import BaggingClassifier
clf=BaggingClassifier(base_estimator=svm_m,n_estimators=10,max_samples=0.8,oob_score=True,random_state=0)
clf.fit(x_train,y_train)
clf.score(x_test,y_test)


C:\Users\Aditya Ware\anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py:640: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\Aditya Ware\anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py:644: RuntimeWarning: invalid value encountered in true_divide
  oob_decision_function = (predictions /


0.8586956521739131

In [28]:
from sklearn.tree import DecisionTreeClassifier
tree_m=DecisionTreeClassifier()
from sklearn.model_selection import cross_val_score
score=cross_val_score(tree_m,x_scale,y,cv=10)
score.mean()


0.7732680363115145

In [31]:
clft=BaggingClassifier(base_estimator=tree_m,n_estimators=10,max_samples=0.8,oob_score=True,random_state=0)
clft.fit(x_train,y_train)
clft.score(x_test,y_test)


C:\Users\Aditya Ware\anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py:640: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\Aditya Ware\anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py:644: RuntimeWarning: invalid value encountered in true_divide
  oob_decision_function = (predictions /


0.8913043478260869